In [24]:
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np


import torch
import torchvision
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from skimage import io, transform
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

In [25]:
import torchvision.transforms as transforms
scale = 28
distortion_scale = 0.3
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.RandomRotation(degrees=[-45, 45]),
    transforms.RandomPerspective(distortion_scale=0.3, p=0.5),
    transforms.Normalize(mean=0.1307, std=0.3081)])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import Subset



train_dataset = MNIST(root='./data', download=True, train=True, transform=transform)
train_subset_indices = list(range(0, len(train_dataset), 2))
train_subset = Subset(train_dataset, train_subset_indices)

test_dataset = MNIST(root='./data', download=True, train=False, transform=transform)
test_subset_indices = list(range(0, len(test_dataset), 2))
test_subset = Subset(test_dataset, test_subset_indices)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>



RuntimeError: Error downloading train-images-idx3-ubyte.gz

In [ ]:
batch = 64

from torch.utils.data import DataLoader
train_loader = DataLoader(train_subset, batch_size=batch, shuffle=True, num_workers=0)
test_loader = DataLoader(test_subset, batch_size=batch, shuffle=True, num_workers=0)

In [ ]:
import math

def feature_map_dim(input_dim, padding, kernel_size, stride):

  output_dim = math.floor((input_dim + 2 * padding - 1 * (kernel_size - 1) - 1) / stride) + 1
  return output_dim

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # 1. Initialize 2 convolution blocks (consists of a convolution layer, an activation function, a MaxPooling layer)

        # Convolution Block 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, stride=1)
        self.mp1 = nn.MaxPool2d(kernel_size=2) # default stride is the same as the kernel size
        self.relu = nn.ReLU()

        # Convolution Block 2 
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=10, kernel_size=3, stride=1)
        self.mp2 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(in_features=250, out_features=100)
        self.fc2 = nn.Linear(in_features=100, out_features=50)

    def forward(self, x):

        outputs = self.conv1(x)
        outputs = self.mp1(outputs)
        outputs = self.relu(outputs)
        outputs = self.conv2(outputs)
        outputs = self.mp2(outputs)
        outputs = self.relu(outputs)

        outputs = self.flatten(outputs)

        outputs = self.relu(self.fc1(outputs))
        outputs = self.relu(self.fc2(outputs))

        return outputs

In [ ]:
cnn = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=1e-4) 
epoch = 10

acc_LIST_CNN = []
loss_LIST_CNN = []

for epoch in range(epoch):
  running_loss = 0.0
  correct = 0
  total = 0
  for inputs, labels in train_loader:
      labels = labels.type(torch.LongTensor)
      inputs, labels = inputs.to(device), labels.to(device)

      optimizer.zero_grad()

      outputs = cnn(inputs)

      predictions = torch.argmax(outputs, dim=1)

      loss = criterion(outputs, labels)

      loss.backward()

      optimizer.step()

      running_loss += loss.item()

      correct += (predictions == labels).sum().detach().cpu().numpy()
      total += len(labels)

  accuracy = (correct/total) * 100
  acc_LIST_CNN.append(accuracy)

  loss_LIST_CNN.append(running_loss / len(train_loader))

  print("The loss for Epoch {} is: {}, Accuracy = {}".format(epoch, running_loss/len(train_loader), accuracy))


In [ ]:
total = 0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.type(torch.LongTensor) 
        images, labels = images.to(device), labels.to(device)

        outputs = cnn(images)

        predictions = torch.argmax(outputs, dim=1)

        correct += (predictions == labels).sum().detach().cpu().numpy()
        total += len(labels)

test_acc_CNN = 100 * correct / total

print(f'Test Accuracy: ' + str(test_acc_CNN.item()))